In [1]:
# install dependencies
!pip install torch
!pip install git+https://github.com/zclaytor/prot

  Using cached torch-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Usin

In [32]:
import os
from tqdm import tqdm

import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from astropy.io import fits
from astroquery.mast import Observations
import lightkurve as lk

from prot import WaveletTransform as WT

In [3]:
Observations.enable_cloud_dataset() # use cloud data when possible
fs = s3fs.S3FileSystem(anon=True)   # read cloud data as though local

INFO: Using the S3 STScI public dataset [astroquery.mast.cloud]


In [97]:
catname = "tess-rot"
cat = pd.read_csv(f"catalogs/{catname}.csv", index_col=0, converters={"sectors": lambda x: x.strip("[]").replace("'","").split(", ")})
cat

,sectors,period,range
TIC,,,
160637,[11],0.389000,NaN
690975,"[7, 34]",6.882000,0.68200
690979,"[7, 34]",6.907150,0.63170
756091,[7],6.304000,NaN
757171,"[7, 34]",4.377050,0.04590
...,...,...,...
2051560493,[24],10.505000,NaN
2051578115,"[18, 19, 24, 25, 26]",1.500688,0.00387
2051652155,"[18, 19, 24, 25, 26]",1.999256,0.02095


In [101]:
if catname == "tess-rot":
    from astropy.table import vstack
    tessobs = [Observations.query_criteria(target_name=cat_i.index.astype(str), project="TESS", dataproduct_type="timeseries", provenance_name="SPOC") \
                   for cat_i in np.array_split(cat, 5)]
    tessobs = vstack(tessobs)
else:
    tessobs = Observations.query_criteria(target_name=cat.index.astype(str), project="TESS", dataproduct_type="timeseries", provenance_name="SPOC")
    
tessobs

/opt/conda/envs/tess/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str4,str10,str4,str4,str7,str10,str1,str52,float64,float64,str10,str14,int64,float64,float64,float64,float64,float64,str1,float64,str48,str1,int64,str48,str1,str79,str6,bool,float64,str9,str9
science,TESS,SPOC,Photometer,TESS,TESS,Optical,26064364,--,tess2020106103520-s0024-0000000026064364-0180-s,353.284731400888,53.0238313161457,timeseries,"Ricker, George",3,58955.29021051,58981.77675398,120.0,600.0,1000.0,--,59001.0,N/A,--,24,CIRCLE ICRS 353.28473140 53.02383132 0.00138889,--,mast:TESS/product/tess2020106103520-s0024-0000000026064364-0180-s_lc.fits,PUBLIC,False,nan,27399490,70383257
science,TESS,SPOC,Photometer,TESS,TESS,Optical,26214389,--,tess2020106103520-s0024-0000000026214389-0180-s,353.961717854258,52.6151235086171,timeseries,"Ricker, George",3,58955.2901662,58981.77671148,120.0,600.0,1000.0,--,59001.0,N/A,--,24,CIRCLE ICRS 353.96171785 52.61512351 0.00138889,--,mast:TESS/product/tess2020106103520-s0024-0000000026214389-0180-s_lc.fits,PUBLIC,False,nan,27402056,70388389
science,TESS,SPOC,Photometer,TESS,TESS,Optical,28639393,--,tess2019198215352-s0014-0000000028639393-0150-s,282.552132743292,34.6138808609646,timeseries,"Ricker, George",3,58682.85554211,58709.70644492,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 282.55213274 34.61388086 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000028639393-0150-s_lc.fits,PUBLIC,False,nan,27429872,70444002
science,TESS,SPOC,Photometer,TESS,TESS,Optical,16316642,--,tess2019198215352-s0014-0000000016316642-0150-s,307.613757230075,41.2236466246169,timeseries,"Ricker, George",3,58682.85506345,58709.70690149,120.0,600.0,1000.0,--,58739.3333334,G022062,--,14,CIRCLE ICRS 307.61375723 41.22364662 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000016316642-0150-s_lc.fits,PUBLIC,False,nan,27433508,70451271
science,TESS,SPOC,Photometer,TESS,TESS,Optical,68487463,--,tess2019198215352-s0014-0000000068487463-0150-s,280.775107860192,35.2282439519806,timeseries,"Ricker, George",3,58682.85545336,58709.70631042,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 280.77510786 35.22824395 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000068487463-0150-s_lc.fits,PUBLIC,False,nan,27434162,70452579
science,TESS,SPOC,Photometer,TESS,TESS,Optical,63368466,--,tess2019226182529-s0015-0000000063368466-0151-s,292.646802277234,42.9696282994431,timeseries,"Ricker, George",3,58710.86605494,58736.9113606,120.0,600.0,1000.0,--,58756.3333334,N/A,--,15,CIRCLE ICRS 292.64680228 42.96962830 0.00138889,--,mast:TESS/product/tess2019226182529-s0015-0000000063368466-0151-s_lc.fits,PUBLIC,False,nan,27481615,70547464
science,TESS,SPOC,Photometer,TESS,TESS,Optical,29806666,--,tess2019198215352-s0014-0000000029806666-0150-s,288.276501053627,30.0461353056403,timeseries,"Ricker, George",3,58682.85600677,58709.70702488,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 288.27650105 30.04613531 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000029806666-0150-s_lc.fits,PUBLIC,False,nan,27431125,70446507
science,TESS,SPOC,Photometer,TESS,TESS,Optical,13332837,--,tess2019198215352-s0014-0000000013332837-0150-s,305.396376454859,38.613279214209,timeseries,"Ricker, George",3,58682.85531793,58709.70705865,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 305.39637645 38.61327921 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000013332837-0150-s_lc.fits,PUBLIC,False,nan,27440587,70465429
science,TESS,SPOC,Photometer,TESS,TESS,Optical,28082086,--,tess2019226182529-s0015-0000000028082086-0151-s,297.181117550584,51.1504785449951,timeseries,"Ricker, George",3,58710.86556257,58736.91121154,120

In [102]:
filtered_obs = tessobs[[str(row["sequence_number"]) in cat.loc[int(row["target_name"]), "sectors"] for row in tessobs]]
filtered_obs

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str4,str10,str4,str4,str7,str10,str1,str52,float64,float64,str10,str14,int64,float64,float64,float64,float64,float64,str1,float64,str48,str1,int64,str48,str1,str79,str6,bool,float64,str9,str9
science,TESS,SPOC,Photometer,TESS,TESS,Optical,26214389,--,tess2020106103520-s0024-0000000026214389-0180-s,353.961717854258,52.6151235086171,timeseries,"Ricker, George",3,58955.2901662,58981.77671148,120.0,600.0,1000.0,--,59001.0,N/A,--,24,CIRCLE ICRS 353.96171785 52.61512351 0.00138889,--,mast:TESS/product/tess2020106103520-s0024-0000000026214389-0180-s_lc.fits,PUBLIC,False,nan,27402056,70388389
science,TESS,SPOC,Photometer,TESS,TESS,Optical,16316642,--,tess2019198215352-s0014-0000000016316642-0150-s,307.613757230075,41.2236466246169,timeseries,"Ricker, George",3,58682.85506345,58709.70690149,120.0,600.0,1000.0,--,58739.3333334,G022062,--,14,CIRCLE ICRS 307.61375723 41.22364662 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000016316642-0150-s_lc.fits,PUBLIC,False,nan,27433508,70451271
science,TESS,SPOC,Photometer,TESS,TESS,Optical,63368466,--,tess2019226182529-s0015-0000000063368466-0151-s,292.646802277234,42.9696282994431,timeseries,"Ricker, George",3,58710.86605494,58736.9113606,120.0,600.0,1000.0,--,58756.3333334,N/A,--,15,CIRCLE ICRS 292.64680228 42.96962830 0.00138889,--,mast:TESS/product/tess2019226182529-s0015-0000000063368466-0151-s_lc.fits,PUBLIC,False,nan,27481615,70547464
science,TESS,SPOC,Photometer,TESS,TESS,Optical,29806666,--,tess2019198215352-s0014-0000000029806666-0150-s,288.276501053627,30.0461353056403,timeseries,"Ricker, George",3,58682.85600677,58709.70702488,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 288.27650105 30.04613531 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000029806666-0150-s_lc.fits,PUBLIC,False,nan,27431125,70446507
science,TESS,SPOC,Photometer,TESS,TESS,Optical,13332837,--,tess2019198215352-s0014-0000000013332837-0150-s,305.396376454859,38.613279214209,timeseries,"Ricker, George",3,58682.85531793,58709.70705865,120.0,600.0,1000.0,--,58739.3333334,N/A,--,14,CIRCLE ICRS 305.39637645 38.61327921 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000013332837-0150-s_lc.fits,PUBLIC,False,nan,27440587,70465429
science,TESS,SPOC,Photometer,TESS,TESS,Optical,63291518,--,tess2019226182529-s0015-0000000063291518-0151-s,292.54455119145,47.7211188105095,timeseries,"Ricker, George",3,58710.86568899,58736.91113821,120.0,600.0,1000.0,--,58756.3333334,N/A,--,15,CIRCLE ICRS 292.54455119 47.72111881 0.00138889,--,mast:TESS/product/tess2019226182529-s0015-0000000063291518-0151-s_lc.fits,PUBLIC,False,nan,27471720,70527674
science,TESS,SPOC,Photometer,TESS,TESS,Optical,63210873,--,tess2019226182529-s0015-0000000063210873-0151-s,292.273693813386,43.6179463958318,timeseries,"Ricker, George",3,58710.86599213,58736.91130916,120.0,600.0,1000.0,--,58756.3333334,N/A,--,15,CIRCLE ICRS 292.27369381 43.61794640 0.00138889,--,mast:TESS/product/tess2019226182529-s0015-0000000063210873-0151-s_lc.fits,PUBLIC,False,nan,27482608,70549450
science,TESS,SPOC,Photometer,TESS,TESS,Optical,12578537,--,tess2019198215352-s0014-0000000012578537-0150-s,304.840467753457,40.8879045312051,timeseries,"Ricker, George",3,58682.85512257,58709.70686597,120.0,600.0,1000.0,--,58739.3333334,G022062,--,14,CIRCLE ICRS 304.84046775 40.88790453 0.00138889,--,mast:TESS/product/tess2019198215352-s0014-0000000012578537-0150-s_lc.fits,PUBLIC,False,nan,27439509,70463273
science,TESS,SPOC,Photometer,TESS,TESS,Optical,17448785,--,tess2019226182529-s0015-0000000017448785-0151-s,308.305256221964,41.2246269148096,timeseries,"Ricker, George",3,58710.86664801,58736.912

In [103]:
# Get associated science products for each Observation
data_products = Observations.get_product_list(filtered_obs) 

# Keep only the science products
lc_prod = Observations.filter_products(data_products, extension="s_lc.fits", productType="SCIENCE")
lc_prod

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str8,str4,str10,str52,str33,str1,str81,str7,str28,str7,str1,str4,str20,str39,str63,int64,str8,str6,int64,str4
27229932,TESS,timeseries,tess2019357164649-s0020-0000000444892321-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000444892321-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000000444892321-0165-s_lc.fits,1926720,27229932,PUBLIC,3,TESS
27231215,TESS,timeseries,tess2019357164649-s0020-0000000360020172-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000360020172-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,G022109_G022148,tess2019357164649-s0020-0000000360020172-0165-s_lc.fits,1926720,27231215,PUBLIC,3,TESS
27231395,TESS,timeseries,tess2019357164649-s0020-0000000051263554-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000051263554-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,G022133,tess2019357164649-s0020-0000000051263554-0165-s_lc.fits,1926720,27231395,PUBLIC,3,TESS
27231616,TESS,timeseries,tess2019357164649-s0020-0000000284155165-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000284155165-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000000284155165-0165-s_lc.fits,1926720,27231616,PUBLIC,3,TESS
27231717,TESS,timeseries,tess2019357164649-s0020-0000000138409870-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000138409870-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000000138409870-0165-s_lc.fits,1926720,27231717,PUBLIC,3,TESS
27232233,TESS,timeseries,tess2019357164649-s0020-0000000233732174-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000233732174-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000000233732174-0165-s_lc.fits,1926720,27232233,PUBLIC,3,TESS
27232558,TESS,timeseries,tess2019357164649-s0020-0000001551704636-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000001551704636-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000001551704636-0165-s_lc.fits,1926720,27232558,PUBLIC,3,TESS
27233026,TESS,timeseries,tess2019357164649-s0020-0000000259098122-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000259098122-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,N/A,tess2019357164649-s0020-0000000259098122-0165-s_lc.fits,1926720,27233026,PUBLIC,3,TESS
27233097,TESS,timeseries,tess2019357164649-s0020-0000000280163882-0165-s,Light curves,S,mast:TESS/product/tess2019357164649-s0020-0000000280163882-0165-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-4.0.27-20200326,G022109_G022148,tess2019357164649-s0020-0000000280163882-0165-s_lc.fits,1926720,27233097,PUBLIC,3,TESS


In [104]:
if catname == "tess-flares":
    product_list = lc_prod
else:
    np.random.seed(88)
    i = np.random.choice(np.arange(len(lc_prod)), 2000, replace=False)
    product_list = lc_prod[i]
    
product_list

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str8,str4,str10,str52,str33,str1,str81,str7,str28,str7,str1,str4,str20,str39,str63,int64,str8,str6,int64,str4
62252374,TESS,timeseries,tess2018349182500-s0006-0000000156933657-0126-s,Light curves,S,mast:TESS/product/tess2018349182500-s0006-0000000156933657-0126-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.20-20201120,N/A,tess2018349182500-s0006-0000000156933657-0126-s_lc.fits,1598400,62252374,PUBLIC,3,TESS
62894966,TESS,timeseries,tess2019058134432-s0009-0000000081859368-0139-s,Light curves,S,mast:TESS/product/tess2019058134432-s0009-0000000081859368-0139-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.20-20201120,G011204,tess2019058134432-s0009-0000000081859368-0139-s_lc.fits,1848960,62894966,PUBLIC,3,TESS
62454109,TESS,timeseries,tess2019032160000-s0008-0000000134545456-0136-s,Light curves,S,mast:TESS/product/tess2019032160000-s0008-0000000134545456-0136-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.19-20201114,G011155_G011127_G011060,tess2019032160000-s0008-0000000134545456-0136-s_lc.fits,1805760,62454109,PUBLIC,3,TESS
61539201,TESS,timeseries,tess2018319095959-s0005-0000000399697011-0125-s,Light curves,S,mast:TESS/product/tess2018319095959-s0005-0000000399697011-0125-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,85b260b13b,G011230,tess2018319095959-s0005-0000000399697011-0125-s_lc.fits,1923840,61539201,PUBLIC,3,TESS
28215179,TESS,timeseries,tess2020351194500-s0033-0000000072426025-0203-s,Light curves,S,mast:TESS/product/tess2020351194500-s0033-0000000072426025-0203-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,000de7cb58,G03251,tess2020351194500-s0033-0000000072426025-0203-s_lc.fits,1892160,28215179,PUBLIC,3,TESS
60841471,TESS,timeseries,tess2018206045859-s0001-0000000410451677-0120-s,Light curves,S,mast:TESS/product/tess2018206045859-s0001-0000000410451677-0120-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.10-20200904,G011060,tess2018206045859-s0001-0000000410451677-0120-s_lc.fits,2039040,60841471,PUBLIC,3,TESS
65151316,TESS,timeseries,tess2019112060037-s0011-0000000418576161-0143-s,Light curves,S,mast:TESS/product/tess2019112060037-s0011-0000000418576161-0143-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.23-20210212,N/A,tess2019112060037-s0011-0000000418576161-0143-s_lc.fits,1984320,65151316,PUBLIC,3,TESS
27666681,TESS,timeseries,tess2020160202036-s0026-0000000264101177-0188-s,Light curves,S,mast:TESS/product/tess2020160202036-s0026-0000000264101177-0188-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,spoc-5.0.3-20200718,G022153,tess2020160202036-s0026-0000000264101177-0188-s_lc.fits,1820160,27666681,PUBLIC,3,TESS
28143544,TESS,timeseries,tess2020324010417-s0032-0000000004289378-0200-s,Light curves,S,mast:TESS/product/tess2020324010417-s0032-0000000004289378-0200-s_lc.fits,SCIENCE,Minimum Recommended Products,LC,--,SPOC,8568991798,G03112,tess2020324010417-s0032-0000000004289378-0200-s_lc.fits,1903680,28143544,PUBLIC,3,TESS


In [105]:
# double check that all products are in the original catalog table
for row in product_list:
    obs_id = row["obs_id"].split("-")
    tic = int(obs_id[2])
    sector = int(obs_id[1][1:])
    if str(sector) not in cat.loc[tic, "sectors"]:
        print("!!", tic, sector)

In [106]:
# Saving this for later
lc_uri = Observations.get_cloud_uris(product_list)
lc_uri

['s3://stpubdata/tess/public/tid/s0001/0000/0001/5039/4126/tess2018206045859-s0001-0000000150394126-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0002/3891/9539/tess2018206045859-s0001-0000000238919539-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0002/6261/3883/tess2018206045859-s0001-0000000262613883-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0002/7223/3699/tess2018206045859-s0001-0000000272233699-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0002/8158/2156/tess2018206045859-s0001-0000000281582156-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0003/0071/0088/tess2018206045859-s0001-0000000300710088-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0003/0081/4115/tess2018206045859-s0001-0000000300814115-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0003/0971/4906/tess2018206045859-s0001-0000000309714906-0120-s_lc.fits',
 's3://stpubdata/tess/public/tid/s0001/0000/0003/4927/4787/tess2

In [107]:
np.savetxt(f"wavelets/{catname}-uris.txt", lc_uri, fmt="%s")

In [ ]:
output_dim = 64

for uri in tqdm(lc_uri):
    with fs.open(uri, "rb") as f:
        with fits.open(f, "readonly") as hdulist:
            # read light curve data
            data = hdulist[1].data
            time = data["TIME"]
    
            # fitler out NaNs
            finite = np.isfinite(time)
            time = time[finite]
            data = data[finite]
    
            # convert to lightkurve object
            lc = lk.TessLightCurve(data, time=time).select_flux("SAP_FLUX")
            lc = lc[lc["QUALITY"] == 0]
    
            # compute wavelet transform
            wt = WT.from_lightcurve(lc, minimum_period=0.01, maximum_period=12)
    
            # pool power
            power = torch.nn.functional.adaptive_avg_pool2d(                                          
                torch.tensor(np.expand_dims(wt.power, 0)), output_size=(output_dim, output_dim))
            power = np.squeeze(power.numpy())
            
            # save output
            outname = os.path.basename(uri).replace("lc.fits", "wt")
            np.save(f"wavelets/{catname}/{outname}.npy", power)
    
            # plot and save
            fig, ax = plt.subplots()
            ax.imshow(power, cmap="binary_r", aspect="equal", interpolation="none", rasterized=True)
            ax.axis("off")
            plt.savefig(f"wavelets/{catname}/{outname}.png", bbox_inches="tight", pad_inches=0)
            plt.close(fig)

 64%|██████▎   | 1273/2000 [31:13<18:42,  1.54s/it]